In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as f
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from sklearn import preprocessing
from tqdm import tqdm
from sklearn.preprocessing import FunctionTransformer
import seaborn as sns 
from scipy import stats
import statsmodels.api as sm

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, BaggingRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVR
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
import h2o
from h2o.automl import H2OAutoML
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.neural_network import MLPRegressor
from scipy.stats import norm
import copy
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor, Pool, metrics, cv
import xgboost as xgb
from scipy.stats import gmean
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error


In [11]:
train_df =  pd.read_csv('n_train.csv')
test_df  =  pd.read_csv('n_test.csv')
for c in train_df.columns:
    if train_df[c].dtypes != 'float64':
        print(c)

startdate
year
quarter
month
week
dayofyear
season
loc_group
climateregions__climateregion


In [12]:
exclude_cols = ['index', 'startdate']
target=["contest-tmp2m-14d__tmp2m"]
main_attrs = [c for c in train_df.columns if c not in exclude_cols and c not in target]
print("Main features:", len(main_attrs))

Main features: 260


In [13]:
df = pd.concat([train_df[main_attrs], test_df[main_attrs]], axis=0)
df.shape

(407088, 260)

In [14]:
df.head(3)

,year,quarter,month,week,dayofyear,season,day_of_year_sin,day_of_year_cos,week_sin,week_cos,month_sin,month_cos,season_sin,season_cos,quarter_sin,quarter_cos,lat,lon,loc_group,climateregions__climateregion,contest-pevpr-sfc-gauss-14d__pevpr,nmme0-tmp2m-34w__cancm30,nmme0-tmp2m-34w__cancm40,nmme0-tmp2m-34w__ccsm30,nmme0-tmp2m-34w__ccsm40,nmme0-tmp2m-34w__cfsv20,nmme0-tmp2m-34w__gfdlflora0,nmme0-tmp2m-34w__gfdlflorb0,nmme0-tmp2m-34w__gfdl0,nmme0-tmp2m-34w__nasa0,nmme0-tmp2m-34w__nmme0mean,contest-wind-h10-14d__wind-hgt-10,nmme-tmp2m-56w__cancm3,nmme-tmp2m-56w__cancm4,nmme-tmp2m-56w__ccsm3,nmme-tmp2m-56w__ccsm4,nmme-tmp2m-56w__cfsv2,nmme-tmp2m-56w__gfdl,nmme-tmp2m-56w__gfdlflora,nmme-tmp2m-56w__gfdlflorb,nmme-tmp2m-56w__nasa,nmme-tmp2m-56w__nmmemean,contest-rhum-sig995-14d__rhum,nmme-prate-34w__cancm3,nmme-prate-34w__cancm4,nmme-prate-34w__ccsm3,nmme-prate-34w__ccsm4,nmme-prate-34w__cfsv2,nmme-prate-34w__gfdl,nmme-prate-34w__gfdlflora,nmme-prate-34w__gfdlflorb,nmme-prate-34w__nasa,nmme-prate-34w__nmmemean,contest-wind-h100-14d__wind-hgt-100,nmme0-prate-56w__cancm30,nmme0-prate-56w__cancm40,nmme0-prate-56w__ccsm30,nmme0-prate-56w__ccsm40,nmme0-prate-56w__cfsv20,nmme0-prate-56w__gfdlflora0,nmme0-prate-56w__gfdlflorb0,nmme0-prate-56w__gfdl0,nmme0-prate-56w__nasa0,nmme0-prate-56w__nmme0mean,nmme0-prate-34w__cancm30,nmme0-prate-34w__cancm40,nmme0-prate-34w__ccsm30,nmme0-prate-34w__ccsm40,nmme0-prate-34w__cfsv20,nmme0-prate-34w__gfdlflora0,nmme0-prate-34w__gfdlflorb0,nmme0-prate-34w__gfdl0,nmme0-prate-34w__nasa0,nmme0-prate-34w__nmme0mean,contest-slp-14d__slp,contest-wind-vwnd-925-14d__wind-vwnd-925,nmme-prate-56w__cancm3,nmme-prate-56w__cancm4,nmme-prate-56w__ccsm3,nmme-prate-56w__ccsm4,nmme-prate-56w__cfsv2,nmme-prate-56w__gfdl,nmme-prate-56w__gfdlflora,nmme-prate-56w__gfdlflorb,nmme-prate-56w__nasa,nmme-prate-56w__nmmemean,contest-pres-sfc-gauss-14d__pres,contest-wind-uwnd-250-14d__wind-uwnd-250,nmme-tmp2m-34w__cancm3,nmme-tmp2m-34w__cancm4,nmme-tmp2m-34w__ccsm3,nmme-tmp2m-34w__ccsm4,nmme-tmp2m-34w__cfsv2,nmme-tmp2m-34w__gfdl,nmme-tmp2m-34w__gfdlflora,nmme-tmp2m-34w__gfdlflorb,nmme-tmp2m-34w__nasa,nmme-tmp2m-34w__nmmemean,contest-prwtr-eatm-14d__prwtr,contest-wind-vwnd-250-14d__wind-vwnd-250,contest-precip-14d__precip,contest-wind-h850-14d__wind-hgt-850,contest-wind-uwnd-925-14d__wind-uwnd-925,contest-wind-h500-14d__wind-hgt-500,cancm30,cancm40,ccsm30,ccsm40,cfsv20,gfdlflora0,gfdlflorb0,gfdl0,nasa0,nmme0mean,elevation__elevation,wind-vwnd-250-2010-1,wind-vwnd-250-2010-2,wind-vwnd-250-2010-3,wind-vwnd-250-2010-4,wind-vwnd-250-2010-5,wind-vwnd-250-2010-6,wind-vwnd-250-2010-7,wind-vwnd-250-2010-8,wind-vwnd-250-2010-9,wind-vwnd-250-2010-10,wind-vwnd-250-2010-11,wind-vwnd-250-2010-12,wind-vwnd-250-2010-13,wind-vwnd-250-2010-14,wind-vwnd-250-2010-15,wind-vwnd-250-2010-16,wind-vwnd-250-2010-17,wind-vwnd-250-2010-18,wind-vwnd-250-2010-19,wind-vwnd-250-2010-20,wind-uwnd-250-2010-1,wind-uwnd-250-2010-2,wind-uwnd-250-2010-3,wind-uwnd-250-2010-4,wind-uwnd-250-2010-5,wind-uwnd-250-2010-6,wind-uwnd-250-2010-7,wind-uwnd-250-2010-8,wind-uwnd-250-2010-9,wind-uwnd-250-2010-10,wind-uwnd-250-2010-11,wind-uwnd-250-2010-12,wind-uwnd-250-2010-13,wind-uwnd-250-2010-14,wind-uwnd-250-2010-15,wind-uwnd-250-2010-16,wind-uwnd-250-2010-17,wind-uwnd-250-2010-18,wind-uwnd-250-2010-19,wind-uwnd-250-2010-20,mjo1d__phase,mjo1d__amplitude,mei__mei,mei__meirank,mei__nip,wind-hgt-850-2010-1,wind-hgt-850-2010-2,wind-hgt-850-2010-3,wind-hgt-850-2010-4,wind-hgt-850-2010-5,wind-hgt-850-2010-6,wind-hgt-850-2010-7,wind-hgt-850-2010-8,wind-hgt-850-2010-9,wind-hgt-850-2010-10,sst-2010-1,sst-2010-2,sst-2010-3,sst-2010-4,sst-2010-5,sst-2010-6,sst-2010-7,sst-2010-8,sst-2010-9,sst-2010-10,wind-hgt-500-2010-1,wind-hgt-500-2010-2,wind-hgt-500-2010-3,wind-hgt-500-2010-4,wind-hgt-500-2010-5,wind-hgt-500-2010-6,wind-hgt-500-2010-7,wind-hgt-500-2010-8,wind-hgt-500-2010-9,wind-hgt-500-2010-10,icec-2010-1,icec-2010-2,icec-2010-3,icec-2010-4,icec-2010-5,icec-2010-6,icec-2010-7,icec-2010-8,i

In [19]:
average_wind_df = df.groupby(['lat', 'lon', 'climateregions__climateregion', 'year', 'season'])['contest-wind-h500-14d__wind-hgt-500'].mean().reset_index()
average_wind_df = average_wind_df.rename(columns={"contest-wind-h500-14d__wind-hgt-500": "average-wind"})
average_wind_df

,lat,lon,climateregions__climateregion,year,season,average-wind
0,0.0,0.833333,0,2014,0,5806.259677
1,0.0,0.833333,0,2014,3,5863.289341
2,0.0,0.833333,0,2015,0,5806.418556
3,0.0,0.833333,0,2015,3,5875.363927
4,0.0,0.833333,0,2016,0,5786.928167
...,...,...,...,...,...,...
4107,1.0,0.866667,9,2015,3,5643.513127
4108,1.0,0.866667,9,2016,0,5395.131167
4109,1.0,0.866667,9,2016,3,5670.608370
4110,1.0,0.866667,9,2022,0,5615.937097


In [21]:
df = df.merge(average_wind_df, how='left', on=['lat', 'lon', 'climateregions__climateregion', 'year', 'season'])

In [22]:
df['wind_diff'] = df['contest-wind-h500-14d__wind-hgt-500'] - df['average-wind']

In [23]:
train_df['wind_diff'] = df[:len(train_df)]['wind_diff']
test_df['wind_diff'] = df[len(train_df):]['wind_diff']

In [24]:
tmp_train = pd.read_csv('./sub_train.csv')
exclude_cols = ['index', 'startdate']
target=["contest-tmp2m-14d__tmp2m"]
main_attrs = [c for c in tmp_train.columns if c not in exclude_cols and c not in target]
print("Main features:", len(main_attrs))

Main features: 59


In [25]:
train_df = train_df[main_attrs]
test_df = test_df[main_attrs]
train_df['wind_diff'] = df[:len(train_df)]['wind_diff']
test_df['wind_diff'] = df[len(train_df):]['wind_diff']

In [28]:
train_df[target[0]] = tmp_train[target[0]]

In [26]:
exclude_cols = ['index', 'startdate']
target=["contest-tmp2m-14d__tmp2m"]
main_attrs = [c for c in train_df.columns if c not in exclude_cols and c not in target]
print("Main features:", len(main_attrs))

Main features: 60


In [29]:
X = train_df[main_attrs].values
y = train_df[target[0]].values

skf = KFold(n_splits=5, random_state=2048, shuffle=True)
skf.get_n_splits(X, y)

train_index_list = []
test_index_list = []
train_x_list = []
val_x_list = []
train_y_list = []
val_y_list = []

for i, (train_index, test_index) in tqdm(enumerate(skf.split(X, y))):
    train_index_list.append(train_index)
    test_index_list.append(test_index)

    train_x_list.append(X[train_index, :])
    val_x_list.append(X[test_index, :])
    train_y_list.append(y[train_index])
    val_y_list.append(y[test_index])    
def train_catboost(x, y):
    print(x.shape, y.shape)
    
    cat_models = []
    for i in tqdm(range(5)):
        train_x, val_x = train_x_list[i], val_x_list[i]
        train_y, val_y = train_y_list[i], val_y_list[i]
        clf = CatBoostRegressor(iterations=5000, verbose=200, task_type="GPU",
                           devices='0:1')
        clf.fit(train_x, train_y, eval_set=[(val_x, val_y)], verbose=True, early_stopping_rounds=100)
        cat_models.append(clf)
    
    return cat_models
cat_models = train_catboost(X, y)

preds = []
for cat_model in tqdm(cat_models):
    preds.append(cat_model.predict(test_df[main_attrs].values))
s = 0
for pred in preds:
    s += pred
s = s/5
df = pd.read_csv('./sample_solution.csv')
df['contest-tmp2m-14d__tmp2m'] = s
df.to_csv('submission_corr_important_v11.csv', index=False)

best_df = pd.read_csv('best_sub.csv')
y_best = best_df[target[0]].values

from numpy import dot
from numpy.linalg import norm

cos_sim = dot(y_best, s)/(norm(y_best)*norm(s))
print("cos_sim with best submission:", cos_sim)

5it [00:00,  5.04it/s]


(375734, 60) (375734,)


  0%|                                                                                                | 0/5 [00:00<?, ?it/s]

Learning rate set to 0.072699
0:	learn: 9.2611105	test: 9.2407165	best: 9.2407165 (0)	total: 7.39ms	remaining: 37s
1:	learn: 8.7047201	test: 8.6864884	best: 8.6864884 (1)	total: 12.6ms	remaining: 31.6s
2:	learn: 8.1902040	test: 8.1729695	best: 8.1729695 (2)	total: 19.6ms	remaining: 32.6s
3:	learn: 7.7021178	test: 7.6853778	best: 7.6853778 (3)	total: 33.7ms	remaining: 42.1s
4:	learn: 7.2543093	test: 7.2392819	best: 7.2392819 (4)	total: 40.4ms	remaining: 40.3s
5:	learn: 6.8374810	test: 6.8238708	best: 6.8238708 (5)	total: 44.8ms	remaining: 37.3s
6:	learn: 6.4604845	test: 6.4487624	best: 6.4487624 (6)	total: 49.2ms	remaining: 35.1s
7:	learn: 6.1016108	test: 6.0896723	best: 6.0896723 (7)	total: 53.3ms	remaining: 33.2s
8:	learn: 5.7726174	test: 5.7605555	best: 5.7605555 (8)	total: 57.5ms	remaining: 31.9s
9:	learn: 5.4677865	test: 5.4573177	best: 5.4573177 (9)	total: 61.8ms	remaining: 30.8s
10:	learn: 5.1839611	test: 5.1740255	best: 5.1740255 (10)	total: 66.1ms	remaining: 30s
11:	learn: 4.92

118:	learn: 1.3800963	test: 1.3851040	best: 1.3851040 (118)	total: 606ms	remaining: 24.9s
119:	learn: 1.3764175	test: 1.3815437	best: 1.3815437 (119)	total: 610ms	remaining: 24.8s
120:	learn: 1.3733820	test: 1.3784576	best: 1.3784576 (120)	total: 615ms	remaining: 24.8s
121:	learn: 1.3707106	test: 1.3759422	best: 1.3759422 (121)	total: 619ms	remaining: 24.7s
122:	learn: 1.3679646	test: 1.3731711	best: 1.3731711 (122)	total: 623ms	remaining: 24.7s
123:	learn: 1.3655785	test: 1.3707708	best: 1.3707708 (123)	total: 628ms	remaining: 24.7s
124:	learn: 1.3602733	test: 1.3654495	best: 1.3654495 (124)	total: 636ms	remaining: 24.8s
125:	learn: 1.3568650	test: 1.3620061	best: 1.3620061 (125)	total: 645ms	remaining: 24.9s
126:	learn: 1.3540295	test: 1.3591804	best: 1.3591804 (126)	total: 649ms	remaining: 24.9s
127:	learn: 1.3517103	test: 1.3568200	best: 1.3568200 (127)	total: 653ms	remaining: 24.9s
128:	learn: 1.3495749	test: 1.3547282	best: 1.3547282 (128)	total: 658ms	remaining: 24.8s
129:	learn

...

4775:	learn: 0.3333413	test: 0.3608490	best: 0.3608490 (4775)	total: 22.3s	remaining: 1.05s
4776:	learn: 0.3333211	test: 0.3608328	best: 0.3608328 (4776)	total: 22.3s	remaining: 1.04s
4777:	learn: 0.3332858	test: 0.3608003	best: 0.3608003 (4777)	total: 22.3s	remaining: 1.04s
4778:	learn: 0.3332577	test: 0.3607797	best: 0.3607797 (4778)	total: 22.4s	remaining: 1.03s
4779:	learn: 0.3332233	test: 0.3607475	best: 0.3607475 (4779)	total: 22.4s	remaining: 1.03s
4780:	learn: 0.3331763	test: 0.3607088	best: 0.3607088 (4780)	total: 22.4s	remaining: 1.02s
4781:	learn: 0.3331389	test: 0.3606694	best: 0.3606694 (4781)	total: 22.4s	remaining: 1.02s
4782:	learn: 0.3331144	test: 0.3606470	best: 0.3606470 (4782)	total: 22.4s	remaining: 1.01s
4783:	learn: 0.3330848	test: 0.3606134	best: 0.3606134 (4783)	total: 22.4s	remaining: 1.01s
4784:	learn: 0.3330555	test: 0.3605878	best: 0.3605878 (4784)	total: 22.4s	remaining: 1s
4785:	learn: 0.3330250	test: 0.3605621	best: 0.3605621 (4785)	total: 22.4s	remainin

4871:	learn: 0.3302277	test: 0.3579206	best: 0.3579206 (4871)	total: 22.7s	remaining: 597ms
4872:	learn: 0.3302091	test: 0.3579071	best: 0.3579071 (4872)	total: 22.7s	remaining: 593ms
4873:	learn: 0.3301908	test: 0.3578901	best: 0.3578901 (4873)	total: 22.8s	remaining: 588ms
4874:	learn: 0.3301588	test: 0.3578592	best: 0.3578592 (4874)	total: 22.8s	remaining: 583ms
4875:	learn: 0.3301391	test: 0.3578392	best: 0.3578392 (4875)	total: 22.8s	remaining: 579ms
4876:	learn: 0.3301251	test: 0.3578276	best: 0.3578276 (4876)	total: 22.8s	remaining: 574ms
4877:	learn: 0.3300929	test: 0.3578033	best: 0.3578033 (4877)	total: 22.8s	remaining: 569ms
4878:	learn: 0.3300703	test: 0.3577859	best: 0.3577859 (4878)	total: 22.8s	remaining: 565ms
4879:	learn: 0.3300433	test: 0.3577619	best: 0.3577619 (4879)	total: 22.8s	remaining: 560ms
4880:	learn: 0.3300037	test: 0.3577250	best: 0.3577250 (4880)	total: 22.8s	remaining: 555ms
4881:	learn: 0.3299691	test: 0.3576881	best: 0.3576881 (4881)	total: 22.8s	remai

4967:	learn: 0.3273565	test: 0.3552416	best: 0.3552416 (4967)	total: 23.1s	remaining: 149ms
4968:	learn: 0.3273288	test: 0.3552183	best: 0.3552183 (4968)	total: 23.1s	remaining: 144ms
4969:	learn: 0.3273044	test: 0.3551926	best: 0.3551926 (4969)	total: 23.1s	remaining: 140ms
4970:	learn: 0.3272824	test: 0.3551747	best: 0.3551747 (4970)	total: 23.2s	remaining: 135ms
4971:	learn: 0.3272574	test: 0.3551480	best: 0.3551480 (4971)	total: 23.2s	remaining: 130ms
4972:	learn: 0.3272193	test: 0.3551147	best: 0.3551147 (4972)	total: 23.2s	remaining: 126ms
4973:	learn: 0.3271918	test: 0.3550952	best: 0.3550952 (4973)	total: 23.2s	remaining: 121ms
4974:	learn: 0.3271678	test: 0.3550749	best: 0.3550749 (4974)	total: 23.2s	remaining: 117ms
4975:	learn: 0.3271441	test: 0.3550541	best: 0.3550541 (4975)	total: 23.2s	remaining: 112ms
4976:	learn: 0.3271006	test: 0.3550125	best: 0.3550125 (4976)	total: 23.2s	remaining: 107ms
4977:	learn: 0.3270648	test: 0.3549785	best: 0.3549785 (4977)	total: 23.2s	remai

100%|████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 25.85it/s]


cos_sim with best submission: 0.9998372874683689
